In [1]:
%load_ext autoreload
%autoreload 2
import pyspark
import lsde2021.download as dl
import lsde2021.aggregate as agg
from lsde2021.types import PathLike
from dateutil.relativedelta import relativedelta
from pyspark.sql import SparkSession
from functools import partial
from pprint import pprint
from pathlib import Path
from typing import Tuple
import pandas as pd
import datetime

In [2]:
MAX_MEMORY = "60G"

spark = SparkSession \
    .builder \
    .appName("EDA") \
    .config("spark.executor.memory", MAX_MEMORY) \
    .config("spark.driver.memory", MAX_MEMORY) \
    .config('spark.driver.maxResultSize', MAX_MEMORY) \
    .config('spark.ui.showConsoleProgress', 'false') \
    .getOrCreate()
sc = spark.sparkContext

21/10/10 13:33:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/10/10 13:33:39 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
21/10/10 13:33:39 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [30]:
def download_wikipedia_sql_dump_handler(
    item: Tuple[Tuple[datetime.date, str, str], str],
    dest: PathLike,
    force: bool = False,
) -> Tuple[datetime.date, PathLike]:
    (date, wiki, table), url = item
    filename = Path(
        "/".join(
            dl.wikimedia_sql_dump_local_file(date, wiki=wiki, table=table)
        )
    )
    destination = dest / filename
    
    def validate_file_func(_destination: PathLike) -> bool:
        ok_file = Path(_destination).with_suffix(".ok")
        if not force and _destination.exists() and ok_file.exists():
            return True
        if not force and _destination.exists():
            ok_file.touch()
            return True
        return False

    return date, dl.download_file(
        url, destination=destination, force=force, validate_file_func=validate_file_func
    )

In [31]:
dest = Path("../hdd/wikipedia_sql_dumps")
tables = ["langlinks", "page", "category", "categorylinks"]

languages = pd.read_csv("./data/languages.csv", index_col="code")
languages.head()

,name,dbname,group
code,,,
ar,Arabic,arwiki,ar
ary,Moroccan Arabic,arywiki,ar
arz,Egyptian Arabic,arzwiki,ar
az,Azerbaijani,azwiki,az
azb,South Azerbaijani,azbwiki,az


In [32]:
downloads = list(dl.wikimedia_sql_dump_urls(
    [datetime.date(2021, 10, 1)], wikis=languages["dbname"], tables=tables))
pprint(downloads)

[((datetime.date(2021, 10, 1), 'arwiki', 'langlinks'),
  'https://dumps.wikimedia.org/arwiki/20211001/arwiki-20211001-langlinks.sql.gz'),
 ((datetime.date(2021, 10, 1), 'arwiki', 'page'),
  'https://dumps.wikimedia.org/arwiki/20211001/arwiki-20211001-page.sql.gz'),
 ((datetime.date(2021, 10, 1), 'arwiki', 'category'),
  'https://dumps.wikimedia.org/arwiki/20211001/arwiki-20211001-category.sql.gz'),
 ((datetime.date(2021, 10, 1), 'arwiki', 'categorylinks'),
  'https://dumps.wikimedia.org/arwiki/20211001/arwiki-20211001-categorylinks.sql.gz'),
 ((datetime.date(2021, 10, 1), 'arywiki', 'langlinks'),
  'https://dumps.wikimedia.org/arywiki/20211001/arywiki-20211001-langlinks.sql.gz'),
 ((datetime.date(2021, 10, 1), 'arywiki', 'page'),
  'https://dumps.wikimedia.org/arywiki/20211001/arywiki-20211001-page.sql.gz'),
 ((datetime.date(2021, 10, 1), 'arywiki', 'category'),
  'https://dumps.wikimedia.org/arywiki/20211001/arywiki-20211001-category.sql.gz'),
 ((datetime.date(2021, 10, 1), 'arywiki',

In [33]:
downloaded = sc.parallelize(downloads, numSlices=4).map(
    partial(
        download_wikipedia_sql_dump_handler,
        dest=dest,
        force=False,
    )
).collect()

downloading file ../hdd/wikipedia_sql_dumps/bxrwiki/20211001/bxrwiki-20211001-page.sql.gz ...
using existing file ../hdd/wikipedia_sql_dumps/arwiki/20211001/arwiki-20211001-langlinks.sql.gz ...
downloading file ../hdd/wikipedia_sql_dumps/enwiki/20211001/enwiki-20211001-categorylinks.sql.gz ...
using existing file ../hdd/wikipedia_sql_dumps/itwiki/20211001/itwiki-20211001-category.sql.gz ...
downloading file ../hdd/wikipedia_sql_dumps/arwiki/20211001/arwiki-20211001-page.sql.gz ...
downloading file ../hdd/wikipedia_sql_dumps/itwiki/20211001/itwiki-20211001-categorylinks.sql.gz ...
using existing file ../hdd/wikipedia_sql_dumps/bxrwiki/20211001/bxrwiki-20211001-category.sql.gz ...
downloading file ../hdd/wikipedia_sql_dumps/bxrwiki/20211001/bxrwiki-20211001-categorylinks.sql.gz ...
using existing file ../hdd/wikipedia_sql_dumps/slwiki/20211001/slwiki-20211001-langlinks.sql.gz ...
downloading file ../hdd/wikipedia_sql_dumps/slwiki/20211001/slwiki-20211001-page.sql.gz ...
using existing fi

using existing file ../hdd/wikipedia_sql_dumps/cawiki/20211001/cawiki-20211001-langlinks.sql.gz ...
downloading file ../hdd/wikipedia_sql_dumps/cawiki/20211001/cawiki-20211001-page.sql.gz ...
using existing file ../hdd/wikipedia_sql_dumps/nowiki/20211001/nowiki-20211001-langlinks.sql.gz ...
downloading file ../hdd/wikipedia_sql_dumps/nowiki/20211001/nowiki-20211001-page.sql.gz ...
using existing file ../hdd/wikipedia_sql_dumps/nowiki/20211001/nowiki-20211001-category.sql.gz ...
downloading file ../hdd/wikipedia_sql_dumps/nowiki/20211001/nowiki-20211001-categorylinks.sql.gz ...
using existing file ../hdd/wikipedia_sql_dumps/cawiki/20211001/cawiki-20211001-category.sql.gz ...
downloading file ../hdd/wikipedia_sql_dumps/cawiki/20211001/cawiki-20211001-categorylinks.sql.gz ...
using existing file ../hdd/wikipedia_sql_dumps/cswiki/20211001/cswiki-20211001-langlinks.sql.gz ...
downloading file ../hdd/wikipedia_sql_dumps/cswiki/20211001/cswiki-20211001-page.sql.gz ...
using existing file ../h

using existing file ../hdd/wikipedia_sql_dumps/enwiki/20211001/enwiki-20211001-category.sql.gz ...
using existing file ../hdd/wikipedia_sql_dumps/frwiki/20211001/frwiki-20211001-category.sql.gz ...
downloading file ../hdd/wikipedia_sql_dumps/frwiki/20211001/frwiki-20211001-categorylinks.sql.gz ...
using existing file ../hdd/wikipedia_sql_dumps/gawiki/20211001/gawiki-20211001-langlinks.sql.gz ...
downloading file ../hdd/wikipedia_sql_dumps/gawiki/20211001/gawiki-20211001-page.sql.gz ...
using existing file ../hdd/wikipedia_sql_dumps/gawiki/20211001/gawiki-20211001-category.sql.gz ...
downloading file ../hdd/wikipedia_sql_dumps/gawiki/20211001/gawiki-20211001-categorylinks.sql.gz ...
using existing file ../hdd/wikipedia_sql_dumps/hiwiki/20211001/hiwiki-20211001-langlinks.sql.gz ...
downloading file ../hdd/wikipedia_sql_dumps/hiwiki/20211001/hiwiki-20211001-page.sql.gz ...
using existing file ../hdd/wikipedia_sql_dumps/hiwiki/20211001/hiwiki-20211001-category.sql.gz ...
downloading file .